In [ ]:
from cnn_lstm_1d import CNNLSTM
from fine_tuned import AdaptedModel
import torch

In [ ]:
# Load pretrained model weights
pretrained_model = CNNLSTM()
pretrained_model.load_state_dict(torch.load("weights/cnn-lstm-3.pth"))

# Freeze CNN and LSTM layers
for param in pretrained_model.cnn.parameters():
    param.requires_grad = False
for param in pretrained_model.lstm.parameters():
    param.requires_grad = False

# Add adapter layers
model = AdaptedModel(pretrained_model, device=torch.device("cuda"))

In [ ]:
# Load the train data
from utils import get_train_test_loaders
train_loader, test_loader = get_train_test_loaders("data/custom.csv")

In [ ]:
# Train the model
model.train_model(train_loader)

In [ ]:
# Test the model
from utils import test_model
test_model(model, test_loader)